# Demo 4: Live Update Image Service

## Share mosaic dataset from Enterprise database to support Live Update

In [2]:
import os
import requests
import getpass
import json
import arcpy

## Generate admin token register Enterprise data store

In [4]:
# Create portal token if Server is federated with ArcGIS Enterprise
portal_sharing_url = "https://valyria.esri.com/portal/sharing/rest"

# Use server admin URL if Server is a standalone ArcGIS Server
server_url = "https://valyria.esri.com/server"
server_admin_url = "https://valyria.esri.com/server/admin"

username = input("Admin username: ")
password = getpass.getpass("Admin password: ")


# Generate publishing token (Federated Server)
gentokenurl = portal_sharing_url + "/generateToken"
params = {"f": "json", "username": username, "password": password, "expiration": "20", "client": "referer", "referer": server_url}

r = requests.post(gentokenurl, data=params, verify=False)
token = json.loads(r.text)["token"]

# Register Enterprise database data store
server_regdatastore = server_admin_url + "/data/registerItem"
print(server_regdatastore)
print(token)

fileshare = {
  "path": "/enterpriseDatabases/rdvmdb03_devsummit2020_1",
  "type": "egdb",
  "id": "d4bcdbb3-8352-4db1-a41b-83a34df0814d",
  "totalRefCount": 0,
  "info": {
    "dataStoreConnectionType": "shared",
    "isManaged": False,
    "connectionString": "ENCRYPTED_PASSWORD=00022e68356230544f423642484e32657541574d4867436477513d3d2a00;SERVER=rdvmdb03;INSTANCE=sde:postgresql:rdvmdb03;DBCLIENT=postgresql;DB_CONNECTION_PROPERTIES=rdvmdb03;DATABASE=sde;USER=jie;VERSION=sde.DEFAULT;AUTHENTICATION_MODE=DBMS"
  }
}

params = {"f": "json", "token": token, "item": json.dumps(fileshare), "referer": server_url}
r = requests.post(server_regdatastore, data=params, verify=False)

print(r.text)

Admin username: admin
Admin password: ········


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'valyria.esri.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://valyria.esri.com/server/admin/data/registerItem
z2X_YEwJJE8rl9cY8nABLiYCOQsJ45u1Niqhe0yG3dK3MXp2r7eNX8Q7aWBmyXYR7pSE54vbAhMjHEt8e28a8X1X1_qCa-OjZDqWMVQI1Cg_ByEyMXc3grBpMyfX4LNzvzKe4tbq01BGFbPZg2AYPfldN4muGQCIj5DL4ATgHuE.


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'valyria.esri.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


{"success":true,"status":"success"}


In [5]:
# Define image service configuration
isdef = {
 "serviceName": "demo_sde_ndfd",
 "type": "ImageServer",
 "description": "",
 "capabilities": "Image, Catalog, Metadata, Mensuration",
 "provider": "ArcObjects11",
 "clusterName": "default",
 "minInstancesPerNode": 0,
 "maxInstancesPerNode": 2,
 "instancesPerContainer": 1,
 "maxWaitTime": 60,
 "maxStartupTime": 300,
 "maxIdleTime": 1800,
 "maxUsageTime": 600,
 "loadBalancing": "ROUND_ROBIN",
 "isolationLevel": "HIGH",
 "configuredState": "STARTED",
 "recycleInterval": 24,
 "recycleStartTime": "00:00",
 "keepAliveInterval": -1,
 "private": False,
 "isDefault": False,
 "maxUploadFileSize": 0,
 "allowedUploadFileTypes": "",
 "properties": {
  "connectionString": "ENCRYPTED_PASSWORD=00022e68356230544f423642484e32657541574d4867436477513d3d2a00;SERVER=rdvmdb03;INSTANCE=sde:postgresql:rdvmdb03;DBCLIENT=postgresql;DB_CONNECTION_PROPERTIES=rdvmdb03;DATABASE=sde;USER=jie;VERSION=sde.DEFAULT;AUTHENTICATION_MODE=DBMS",
  "raster": "ndfd_4days",
  "cacheControlMaxAge": "43200",
  "colormapToRGB": "false",
  "defaultResamplingMethod": "1",
 },
 "extensions": [],
 "frameworkProperties": {},
 "datasets": []
}

In [6]:
server_createservice = server_admin_url + "/services/createService"
print(server_createservice)
print(token)

params = {"f": "json", "token": token, "service": json.dumps(isdef), "referer": server_url}
r = requests.post(server_createservice, data=params, verify=False)

print(r.text)

https://valyria.esri.com/server/admin/services/createService
z2X_YEwJJE8rl9cY8nABLiYCOQsJ45u1Niqhe0yG3dK3MXp2r7eNX8Q7aWBmyXYR7pSE54vbAhMjHEt8e28a8X1X1_qCa-OjZDqWMVQI1Cg_ByEyMXc3grBpMyfX4LNzvzKe4tbq01BGFbPZg2AYPfldN4muGQCIj5DL4ATgHuE.


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'valyria.esri.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


{"status":"success"}


## Add new Raster to mosaic dataset without stopping service

In [7]:
mdpath = "\\\\valyria\\Z__DevSummit2020\\Demo\\ManageRaster\\rdvmdb03.sde\\sde.jie.ndfd_4days"

# Remove the old records
arcpy.management.RemoveRastersFromMosaicDataset(mdpath, "stdtime < timestamp '2020-03-21 00:00:00'")

<Result '\\\\valyria\\Z__DevSummit2020\\Demo\\ManageRaster\\rdvmdb03.sde\\sde.jie.ndfd_4days_1'>

In [8]:
# NDFD temporature forecast for the next 4 days from 3/19/20
data = r"\\valyria\Z__DevSummit2020\Demo\ManageRaster\ndfd\ds.temp__031520.bin"
rastertype = "GRIB"
# Note: auxillary input option allows configuration of raster type settings
auxoption = "Variable 'T@HTGL'"

# Add data to mosaic dataset, can either add single dataset or the entire folder. 
arcpy.management.AddRastersToMosaicDataset(mdpath, rastertype, data, aux_inputs=auxoption, estimate_statistics="ESTIMATE_STATISTICS")

<Result '\\\\valyria\\Z__DevSummit2020\\Demo\\ManageRaster\\rdvmdb03.sde\\sde.jie.ndfd_4days_1'>